In [10]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import vk
import numpy as np

Скачиваем с хтмл страницы первую тысячу человек

Собираем у них ссылки и ФИО

После отбираем уникальные фамилии и имена, чтобы искать больше людей

Веб-скрапим страницы и получаем ссылки и имена

In [11]:
with open('labeled name.html', 'r') as f:

    contents = f.read()

    soup = BeautifulSoup(contents, 'html.parser')

#     print(soup.prettify())

In [13]:
df = pd.DataFrame(columns = ['link', 'id', 'name', 'last_name', 'city', 'education', 'interests', 'music', 'movies', 'tv', 'books', 'games', 'can_see_audio'])

info = soup.find_all('div', class_ = 'people_row search_row clear_fix')

for item in info:
    div = item.find_all('div', class_ = 'labeled name')
    a_class = div[0].find_all('a')
#     print(i)
#     print(type(a_class[0]))
    link = a_class[0].get('href')
    spl = a_class[0].text.split(' ')
    name = spl[0]
    last_name = spl[1]
#     name = str(a_class[0].find_all('img')[0]).split("\"")[1]
    
    test = item.find_all('div', class_ = 'labeled')
    if (len(test) > 1):
        if ('Россия' in test[1].text):
            city = test[1].text
        else:
            city = None
    else:
        city = None
    df = df.append({'link': link, 'name': name, 'last_name': last_name, 'city': city}, ignore_index=True)
    
df.to_excel('init_dataset.xlsx')

In [15]:

print(len(np.unique(df['name'])))

print(len(np.unique(df['last_name'])))

391
870


In [19]:
d = {'name': np.unique(df['name'])}
f = {'name': np.unique(df['last_name'])}
unique_names = pd.DataFrame(data = d)
unique_lasts = pd.DataFrame(data = f)
# unique_names = unique_names.append(np.unique(df['last_name']))
unique_values = pd.concat([unique_names, unique_lasts], ignore_index=True)
unique_values.to_excel('unique_names.xlsx')